In [1]:
import random 
import numpy as np
from package.Data_sim import LinearSEMSimulator
from package.pdbn_partial import admm_dy_personal as adp
from package.utils import opt_boundary
from package.Metric import mea_acc

import matplotlib.pyplot as plt

In [2]:
#import the GT matrix
file_path = 'varying_dim/personalized_ad10.npz'
matrix = np.load(file_path)
W = matrix['GT_W']
A = matrix['GT_A']
num_expreiments = 5

tpr, tpr_1 = np.zeros(num_expreiments), np.zeros(num_expreiments)
fdr, fdr_1 = np.zeros(num_expreiments), np.zeros(num_expreiments)
shd, shd_1 = np.zeros(num_expreiments), np.zeros(num_expreiments)

for i in range(num_expreiments):
    print('Experiment number:', i)
    seed = 67
    np.random.seed(seed)

    K, d_vars = W.shape[0], W.shape[1]

    n_samples = 60
    lag = 1 
    K, d_vars = W.shape[0], W.shape[1]

    sem_type = 'gauss'
    noise_scale = 1

    X_agent = np.zeros((K, n_samples, d_vars))
    Y_agent = np.zeros((K, n_samples, lag * d_vars))
    # print(X_agent.shape, Y_agent.shape)

    for k in range(K):
        simulator = LinearSEMSimulator(W[k], noise_scale)
        X = simulator.simulate(n_samples, sem_type)

        # Preprocess data for Federated Learning
        X_up = simulator.add_inter_slice_effects(X, A[k])
        Y = simulator.create_lagged_version(X, lag)

        # Add inter-slice effects and reshape
        X_agent[k] = X_up
        Y_agent[k] = Y


    bnds = opt_boundary(X_up, Y, d_vars)

    X_agent = np.array(X_agent, dtype=np.float64)
    Y_agent = np.array(Y_agent, dtype=np.float64)

    mu = 0.1
    lambda_w = 0.1
    lambda_a = 0.1
    w_threshold=0.3
    a_threshold=0.3

    _, _, W_k, A_k=  adp(X_agent, Y_agent, bnds, mu = mu, lambda_w=lambda_w, lambda_a=lambda_a,partial = None0, w_threshold=w_threshold, a_threshold=a_threshold)
    # W_est, A_est = adn(X_agent, Y_agent, bnds, lambda_w=lambda_w, lambda_a=lambda_a, w_threshold=w_threshold, a_threshold=a_threshold)


    shd_w, tpr_w, fdr_w = np.zeros(K), np.zeros(K), np.zeros(K)
    shd_a, tpr_a, fdr_a = np.zeros(K), np.zeros(K), np.zeros(K)


    for k in range(K):
        shd_w[k], fdr_w[k], tpr_w[k] = mea_acc(W_k[k],W[k])
        shd_a[k], fdr_a[k], tpr_a[k] = mea_acc(A_k[k],A[k])

    

    shd[i] = np.mean(shd_w)
    tpr[i] = np.mean(tpr_w)
    fdr[i] = np.mean(fdr_w)
    shd_1[i] = np.mean(shd_a)
    tpr_1[i] = np.mean(tpr_a)
    fdr_1[i] = np.mean(fdr_a)


Experiment number: 0
Experiment number: 1
Experiment number: 2
Experiment number: 3
Experiment number: 4


In [3]:
print('SHD:', np.mean(shd))
print('TPR:', np.mean(tpr))
print('FDR:', np.mean(fdr))
print('SHD:', np.mean(shd_1))
print('TPR:', np.mean(tpr_1))
print('FDR:', np.mean(fdr_1))

SHD: 13.666666666666666
TPR: 0.6357600732600733
FDR: 0.5451388888888887
SHD: 8.333333333333334
TPR: 0.5676224968988127
FDR: 0.24299450549450552


## result for all sparse:

d = 5 (2 of 5)

hyper for both are 0.3 and sample = 30
 
SHD: 4.416666666666666

TPR: 0.6138888888888888

FDR: 0.7161904761904763

SHD: 2.8499999999999996

TPR: 0.8055952380952383

FDR: 0.35714285714285715


d = 10 (4 of 5)



d= 15/20: 



## result for sparse and dense 

d = 5 (half with 2 of 5 and half with 4 of 5)

hyper for 5 and 10: 0.1, 0.1 0.1 

SHD: 4.866666666666666

TPR: 0.6054761904761905

FDR: 0.44749999999999995

SHD: 3.5666666666666664

TPR: 0.6767869167869168

FDR: 0.24174603174603174

d = 10 (half with 4 of 10 and half with 8 of 10)

SHD: 23.9

TPR: 0.5946212865545885

FDR: 0.6259767652750108

SHD: 16.03333333333333

TPR: 0.3728226346710328

FDR: 0.33553030303030307


d = 15 (half with 7 of 15 and half with 12 of 15)

hyper: 0.2, 0.01, 0.01 

SHD: 51.73333333333333

TPR: 0.6622265267934067

FDR: 0.5295692527453364

SHD: 26.7

TPR: 0.2947849736585529

FDR: 0.3061553724053724